In [ ]:
# Dependencies
import numpy as np
import requests
import json
import pandas as pd
from datetime import datetime as dt

In [ ]:
# Incorporate list of points
grids = "GRIDS.xlsx"
grids_df = pd.read_excel(grids)
grids_df.head()

In [ ]:
#Create URL list

urls = []
posts = []

for g in range(0,len(grids_df),1):
    
    try:
        
        office = "OKX"
        
        gridX = grids_df['gridX'][g]
        gridY = grids_df['gridY'][g]
        
        urls.append(f'https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}')
        posts.append(requests.get(urls[g]).json())
    
    except:
        pass
    
    print(f'{urls[g]} : succesfully documented')
    print("-----------------------------")
    
print("URLs Logged")
print("-----------------------------")

In [ ]:
# URL for GET requests to retrieve data

base_urls = []

for g in range(0,len(urls),1):

    try:

        base_urls.append(urls[g])

    except:
        pass

#Set responses

responses = []

for b in range(0,len(base_urls),1):

    try:
            
        responses.append(requests.get(base_urls[b]).json())

    except:
        pass

#Align dictionaries

weather = []

for r in range(0,len(responses),1):

    try:

        weather.append(responses[r]['properties'])

    except:
        pass

In [ ]:
#Export API Scrape

dfs = []

#Establish Conversion Functions

def angle_to_direction(angle):
      if 337.5 <= angle < 22.5:
        return "N"
      elif 22.5 <= angle < 67.5:
        return "NE"
      elif 67.5 <= angle < 112.5:
        return "E"
      elif 112.5 <= angle < 157.5:
        return "SE"
      elif 157.5 <= angle < 202.5:
        return "S"
      elif 202.5 <= angle < 247.5:
        return "SW"
      elif 247.5 <= angle < 292.5:
        return "W"
      else:
          return "NW" 
      
def kmh_to_mph(kmh):
    mph = kmh/1.60934
    return mph

def mm_to_inches(mm):
    inches = mm * 0.0393701
    return inches

def m_to_ft(m):
    feet = m * 3.28084
    return feet

#Begin Parsing Loop   
for z in range(0,len(weather),1):
        
        #Create Wind Chill Extract
        w_indices = weather[z]['windChill']['values']
        w_indices_df = pd.DataFrame(w_indices)
        w_input_string = w_indices_df['validTime'].str.split('T')
        w_data = w_input_string
        w_dates = [item[0] for item in w_data]
        w_date_times = [pd.to_datetime(date) for date in w_dates]

        try:
            w_df = pd.DataFrame({'Date': w_date_times, 'values': (w_indices_df['value'])*(9/5)+32, 'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass

        w_df_min = w_df.resample('d', on='Date').min()

        #Create Heat Index Extract
        h_indices = weather[z]['heatIndex']['values']
        h_indices_df = pd.DataFrame(h_indices)
        h_input_string = h_indices_df['validTime'].str.split('T')
        h_data = h_input_string  
        h_dates = [item[0] for item in h_data]  
        h_date_times = [pd.to_datetime(date) for date in h_dates]
        try: 
            h_df = pd.DataFrame({'Date': h_date_times, 'values': (h_indices_df['value'])*(9/5)+32,'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        h_df_max = h_df.resample('d', on='Date').max()

        #Create Air Temperature Extract
        t_indices = weather[z]['temperature']['values']
        t_indices_df = pd.DataFrame(t_indices)
        t_input_string = t_indices_df['validTime'].str.split('T')
        t_data = t_input_string  
        t_dates = [item[0] for item in t_data]  
        t_date_times = [pd.to_datetime(date) for date in t_dates]
        try: 
            t_df = pd.DataFrame({'Date': t_date_times, 'values': (t_indices_df['value'])*(9/5)+32,'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        t_df_avg = t_df.resample('d', on='Date').mean()

        #Create Dewpoint Extract
        d_indices = weather[z]['dewpoint']['values']
        d_indices_df = pd.DataFrame(d_indices)
        d_input_string = d_indices_df['validTime'].str.split('T')
        d_data = d_input_string  
        d_dates = [item[0] for item in d_data]  
        d_date_times = [pd.to_datetime(date) for date in d_dates]
        try: 
            d_df = pd.DataFrame({'Date': d_date_times, 'values': (d_indices_df['value'])*(9/5)+32,'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        d_df_avg = d_df.resample('d', on='Date').mean()

        #Create Relative Humidity
        rh_indices = weather[z]['relativeHumidity']['values']
        rh_indices_df = pd.DataFrame(rh_indices)
        rh_input_string = rh_indices_df['validTime'].str.split('T')
        rh_data = rh_input_string  
        rh_dates = [item[0] for item in rh_data]  
        rh_date_times = [pd.to_datetime(date) for date in rh_dates]
        try: 
            rh_df = pd.DataFrame({'Date': rh_date_times, 'values': rh_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        rh_df_avg = rh_df.resample('d', on='Date').mean()

        #Create Max Apparent Temperature Extract
        maxap_indices = weather[z]['apparentTemperature']['values']
        maxap_indices_df = pd.DataFrame(maxap_indices)
        maxap_input_string = maxap_indices_df['validTime'].str.split('T')
        maxap_data = maxap_input_string  
        maxap_dates = [item[0] for item in maxap_data]  
        maxap_date_times = [pd.to_datetime(date) for date in maxap_dates]
        try: 
            maxap_df = pd.DataFrame({'Date': maxap_date_times, 'values': (maxap_indices_df['value'])*(9/5)+32,'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        maxap_df_max = maxap_df.resample('d', on='Date').max()

        #Create Min Apparent Temperature Extract
        minap_indices = weather[z]['apparentTemperature']['values']
        minap_indices_df = pd.DataFrame(minap_indices)
        minap_input_string = minap_indices_df['validTime'].str.split('T')
        minap_data = minap_input_string  
        minap_dates = [item[0] for item in minap_data]  
        minap_date_times = [pd.to_datetime(date) for date in minap_dates]
        try: 
            minap_df = pd.DataFrame({'Date': minap_date_times, 'values': (minap_indices_df['value'])*(9/5)+32,'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        minap_df_max = minap_df.resample('d', on='Date').min()

        #Create Sky Cover Extract
        s_indices = weather[z]['skyCover']['values']
        s_indices_df = pd.DataFrame(s_indices)
        s_input_string = s_indices_df['validTime'].str.split('T')
        s_data = s_input_string  
        s_dates = [item[0] for item in s_data]  
        s_date_times = [pd.to_datetime(date) for date in s_dates]
        try: 
            s_df = pd.DataFrame({'Date': s_date_times, 'values': s_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        s_df_avg = s_df.resample('d', on='Date').mean()


        #Create Wind Direction Extract
        wind_indices = weather[z]['windDirection']['values']
        wind_indices_df = pd.DataFrame(wind_indices)
        wind_input_string = wind_indices_df['validTime'].str.split('T')
        wind_data = wind_input_string  
        wind_dates = [item[0] for item in wind_data]  
        wind_date_times = [pd.to_datetime(date) for date in wind_dates]
        try: 
            wind_df = pd.DataFrame({'Date': wind_date_times, 'values': wind_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        wind_df_avg = wind_df.resample('d', on='Date').mean()

        #Create Wind Speed Extract
        speed_indices = weather[z]['windSpeed']['values']
        speed_indices_df = pd.DataFrame(speed_indices)
        speed_input_string = speed_indices_df['validTime'].str.split('T')
        speed_data = speed_input_string  
        speed_dates = [item[0] for item in speed_data]  
        speed_date_times = [pd.to_datetime(date) for date in speed_dates]
        try: 
            speed_df = pd.DataFrame({'Date': speed_date_times, 'values': speed_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        speed_df_max = speed_df.resample('d', on='Date').max()

        #Create Wind Gust Extract
        gust_indices = weather[z]['windGust']['values']
        gust_indices_df = pd.DataFrame(gust_indices)
        gust_input_string = gust_indices_df['validTime'].str.split('T')
        gust_data = gust_input_string  
        gust_dates = [item[0] for item in gust_data]  
        gust_date_times = [pd.to_datetime(date) for date in gust_dates]
        try: 
            gust_df = pd.DataFrame({'Date': gust_date_times, 'values': gust_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        gust_df_max = gust_df.resample('d', on='Date').max()

        #Create Probability of Precipitation Extract
        probp_indices = weather[z]['probabilityOfPrecipitation']['values']
        probp_indices_df = pd.DataFrame(probp_indices)
        probp_input_string = probp_indices_df['validTime'].str.split('T')
        probp_data = probp_input_string  
        probp_dates = [item[0] for item in probp_data]  
        probp_date_times = [pd.to_datetime(date) for date in probp_dates]
        try: 
            probp_df = pd.DataFrame({'Date': probp_date_times, 'values': probp_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        probp_df_max = probp_df.resample('d', on='Date').max()

        #Create Quantitative Precipitation Extract
        qp_indices = weather[z]['quantitativePrecipitation']['values']
        qp_indices_df = pd.DataFrame(qp_indices)
        qp_input_string = qp_indices_df['validTime'].str.split('T')
        qp_data = qp_input_string  
        qp_dates = [item[0] for item in qp_data]  
        qp_date_times = [pd.to_datetime(date) for date in qp_dates]
        try: 
            qp_df = pd.DataFrame({'Date': qp_date_times, 'values': qp_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        qp_df_max = qp_df.resample('d', on='Date').max()

        #Create Quantitative Precipitation Extract
        ice_indices = weather[z]['iceAccumulation']['values']
        ice_indices_df = pd.DataFrame(ice_indices)
        ice_input_string = ice_indices_df['validTime'].str.split('T')
        ice_data = ice_input_string  
        ice_dates = [item[0] for item in ice_data]  
        ice_date_times = [pd.to_datetime(date) for date in ice_dates]
        try: 
            ice_df = pd.DataFrame({'Date': ice_date_times, 'values': ice_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        ice_df_max = qp_df.resample('d', on='Date').max()

        #Create Quantitative Precipitation Extract
        snow_indices = weather[z]['snowfallAmount']['values']
        snow_indices_df = pd.DataFrame(snow_indices)
        snow_input_string = snow_indices_df['validTime'].str.split('T')
        snow_data = snow_input_string  
        snow_dates = [item[0] for item in snow_data]  
        snow_date_times = [pd.to_datetime(date) for date in snow_dates]
        try: 
            snow_df = pd.DataFrame({'Date': snow_date_times, 'values': snow_indices_df['value'],'lat': weather[z]['gridX'], 'long': weather[z]['gridY']})
        except:
            pass 

        snow_df_max = qp_df.resample('d', on='Date').max()

        #Begin 208 Loop
        dates = []
        temperature = []
        maxtemp = []
        mintemp = []
        dewpoint = []
        humidity = []
        app_max = []
        app_min = []
        heatindex = []
        windchill=[]
        skycover = []
        wind_direction = []
        speed = []
        gust = []
        prob_p = []
        quant_p = []
        ice = []
        snow = []
        lightning = []
        wave = []
        #Probability of Thunder?
        #Hazards?
        #Attributes for Weather Conditions and Generic Merge?

        for i in range(0,8,1):
            
            try:
                dates.append(weather[z]['maxTemperature']['values'][i]['validTime'])
                #Adjust Arrays for 0 Date
                if i==8:
                    maxtemp.append('N/A')
                else:
                    maxtemp.append((weather[z]['maxTemperature']['values'][i]['value'])*(9/5)+32)
                if i==0:
                    mintemp.append('N/A')
                else:
                    mintemp.append((weather[z]['minTemperature']['values'][i]['value'])*(9/5)+32)
            except:
                pass

            #All Other Parsing Appends
            try:
                temperature.append((round(t_df_avg['values'][i])))
            except:
                temperature.append(0)
            try:
                dewpoint.append((round(d_df_avg['values'][i])))
            except:
                dewpoint.append(0)
            try:
                humidity.append((round(rh_df_avg['values'][i])))
            except:
                humidity.append(0)
            try:
                app_max.append(maxap_df_max['values'][i])
            except:
                app_max.append(0)
            try:
                app_min.append(minap_df_max['values'][i])
            except:
                app_min.append(0)
            try:
                heatindex.append(h_df_max['values'][i])
            except:
                heatindex.append(0)
            try:
                windchill.append(w_df_min['values'][i])
            except:
                windchill.append(0)
            try:
                skycover.append((round(s_df_avg['values'][i])))
            except:
                skycover.append(0)
            try:
                angle = wind_df_avg['values'][i] 
                direction = angle_to_direction(angle)
                wind_direction.append(direction)
            except:
                wind_direction.append(0)  
            try:
                kmh = speed_df_max['values'][i] 
                wind_speed = kmh_to_mph(kmh)
                speed.append((round(wind_speed)))
            except:
                speed.append(0)
            try:
                kmh = gust_df_max['values'][i] 
                wind_gust = kmh_to_mph(kmh)
                gust.append((round(wind_gust)))
            except:
                gust.append(0)
            try:
                prob_p.append((round(probp_df_max['values'][i])))
            except:
                prob_p.append(0)
            try:
                if i < 4:
                    mm = qp_df_max['values'][i] 
                    quant = mm_to_inches(mm)
                    quant_p.append((round(quant)))
                else:
                    quant_p.append('N/A')
            except:
                pass
            try:
                if i < 4:
                    mm = ice_df_max['values'][i] 
                    acc = mm_to_inches(mm)
                    ice.append((round(acc)))
                else:
                    ice.append('N/A')
            except:
                pass
            try:
                if i < 4:
                    mm = snow_df_max['values'][i] 
                    amt = mm_to_inches(mm)
                    snow.append((round(amt)))
                else:
                    snow.append('N/A')
            except:
                pass
            try:
                if i==0:
                    lightning.append(weather[z]['lightningActivityLevel']['values'][i]['value'])
                else:
                    lightning.append('N/A')
            except:
                pass
            try:
                if i==0:
                    m = weather[z]['waveHeight']['values'][i]['value']
                    water = m_to_ft(m)
                    wave.append((round(water)))
                else:
                    wave.append('N/A')
            except:
                pass

            list_trsf = {
            
            'URL' : base_urls[z],
            'Latitude': weather[z]['gridX'],
            'Longitude': weather[z]['gridY'],
            'Date': dates,
            'Temperature (Avg F)': temperature,
            'Max Temperature (F)': maxtemp,
            'Min Temperature (F)': mintemp,
            'Dewpoint (Avg F)': dewpoint,
            'Relative Humidity (Avg %)': humidity,
            "Apparent Temp (Max F)": app_max,
            "Apparent Temp (Min F)": app_min,
            'Heat Index (Max F)': heatindex,
            'Wind Chill (Min F)': windchill,
            'Sky Cover (Avg %)': skycover,
            'Wind Direction': wind_direction,
            'Wind Speed (Max MPH)': speed,
            'Wind Gust (Max MPH)': gust,
            'Probability of Precip. (%)': prob_p,
            'Quantiative Precip. (Max in)': quant_p,
            'Ice Accumulation (Max in)': ice,
            'Snowfall Amount (Max in)': snow,
            'Lightning Activity Level': lightning,
            'Wave Height (ft)': wave
        }

        weather_df_draft = pd.DataFrame(data=list_trsf)
        weather_df_fill = weather_df_draft.fillna("N/A")

        dfs.append(weather_df_fill)

        print("-----------------------------")
        print(f"URL: {base_urls[z]} : logged")
        #print(dfs[z])
        #print("-----------------------------")

In [ ]:
#Set Bucket for Updates Dataframes

weather_dfs = []

#Extract Extra Weather Attributes
def km_to_mi(km):
    mi = km*0.621371
    return mi

coverage = []
weather_y = []
intensity = []
visibility = []
weather_dates = []

for z in range(0,len(weather),1):

    weather_x = weather[z]['weather']['values']

    for x in range(0,len(weather_x),1):

        try:
            coverage.append(weather_x[x]['value'][0]['coverage'].replace("_"," "))
        except:
            coverage.append(weather_x[x]['value'][0]['coverage'])
        try:
            weather_y.append(weather_x[x]['value'][0]['weather'].replace("_"," "))
        except:
            weather_y.append(weather_x[x]['value'][0]['weather'])
        try:
            km = weather_x[x]['value'][0]['visibility']['value']
            viz = km_to_mi(km)
            visibility.append((round(viz)))
        except:
            visibility.append('0')
            
        intensity.append(weather_x[x]['value'][0]['intensity'])
        weather_dates.append(weather_x[x]['validTime'])

        weather_list_trsf = {
            'URL': base_urls[z],
            'Latitude': weather[z]['gridX'],
            'Longitude': weather[z]['gridY'],
            'Date': weather_dates,
            'Coverage': coverage,
            'Condition': weather_y,
            'Intensity': intensity,
            'Visibility (mi)': visibility  
        }

    # print("-----------------------------")
    # print(f"URL: {base_urls[z]} : logged")

    conditions_df_draft = pd.DataFrame(data=weather_list_trsf)
    conditions_df_fill = conditions_df_draft.fillna("N/A")

    weather_dfs.append(conditions_df_fill)

In [ ]:

combined_df = pd.concat(dfs)
date_conv = combined_df['Date'].str.split('T', n=1, expand=True)
combined_df['Date'] = date_conv[0]

weather_df = pd.concat(weather_dfs)
x_date_conv = weather_df['Date'].str.split('+', n=1, expand=True)
weather_df['Date'] = x_date_conv[0]

# Export the combined DataFrame to a single CSV file
weather_df.to_csv('nyc_weather_conditions.csv', index=False)
combined_df.to_csv('nyc_forecasts.csv', index=False)